In [1]:
import pandas as pd
import numpy as np
from nba_api.stats import endpoints

In [9]:
pd.set_option('display.max_columns', None)
temp = endpoints.leaguestandings.LeagueStandings(season=["2009-10"]).standings.get_data_frame()
temp

,LeagueID,SeasonID,TeamID,TeamCity,TeamName,Conference,ConferenceRecord,PlayoffRank,ClinchIndicator,Division,DivisionRecord,DivisionRank,WINS,LOSSES,WinPCT,LeagueRank,Record,HOME,ROAD,L10,Last10Home,Last10Road,OT,ThreePTSOrLess,TenPTSOrMore,LongHomeStreak,strLongHomeStreak,LongRoadStreak,strLongRoadStreak,LongWinStreak,LongLossStreak,CurrentHomeStreak,strCurrentHomeStreak,CurrentRoadStreak,strCurrentRoadStreak,CurrentStreak,strCurrentStreak,ConferenceGamesBack,DivisionGamesBack,ClinchedConferenceTitle,ClinchedDivisionTitle,ClinchedPlayoffBirth,EliminatedConference,EliminatedDivision,AheadAtHalf,BehindAtHalf,TiedAtHalf,AheadAtThird,BehindAtThird,TiedAtThird,Score100PTS,OppScore100PTS,OppOver500,LeadInFGPCT,LeadInReb,FewerTurnovers,PointsPG,OppPointsPG,DiffPointsPG,vsEast,vsAtlantic,vsCentral,vsSoutheast,vsWest,vsNorthwest,vsPacific,vsSouthwest,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,PreAS,PostAS
0,00,22009,1610612739,Cleveland,Cavaliers,East,38-14,1,- e,Central,12-4,1,61,21,0.744,0,61-21,35-6,26-15,4-6,8-2,5-5,2-2,7-8,32-5,11,W 11,5,W 5,13,4,-2,L 2,-4,L 4,-4,L 4,0.0,0.0,1,1,1,0,0,47-8,13-12,1-1,49-8,11-13,1-0,45-5,14-11,31-15,55-8,46-13,25-4,102.1,95.6,6.5,38-14,15-3,12-4,11-7,23-7,7-3,10-0,6-4,12-3,8-3,13-2,2-5,None,None,None,None,None,2-2,10-3,14-3,43-11,18-10
1,00,22009,1610612747,Los Angeles,Lakers,West,35-17,1,- w,Pacific,13-3,1,57,25,0.695,0,57-25,34-7,23-18,4-6,8-2,5-5,4-1,11-7,27-13,11,W 11,5,W 5,11,3,1,W 1,-1,L 1,-1,L 1,0.0,0.0,1,1,1,0,0,41-7,15-18,1-0,45-4,10-21,2-0,40-5,15-16,29-19,46-2,35-10,33-8,101.7,97.0,4.7,22-8,8-2,8-2,6-4,35-17,11-7,13-3,11-7,12-5,8-4,9-6,3-4,None,None,None,None,None,1-1,12-2,12-3,41-13,16-12
2,00,22009,1610612742,Dallas,Mavericks,West,33-19,2,- sw,Southwest,10-6,1,55,27,0.671,0,55-27,28-13,27-14,8-2,6-4,8-2,5-3,9-2,21-14,8,W 8,5,W 5,13,3,2,W 2,5,W 5,5,W 5,2.0,0.0,0,1,1,0,0,34-4,18-23,3-0,44-3,10-24,1-0,35-7,21-19,28-20,42-3,24-8,37-11,102.0,99.3,2.7,22-8,6-4,9-1,7-3,33-19,9-9,14-4,10-6,8-7,9-4,11-4,5-2,None,None,None,None,None,2-1,11-4,9-5,32-20,23-7
3,00,22009,1610612753,Orlando,Magic,East,39-13,2,- se,Southeast,10-6,1,59,23,0.720,0,59-23,34-7,25-16,9-1,9-1,8-2,2-0,3-4,39-6,7,W 7,8,W 8,8,4,7,W 7,2,W 2,6,W 6,2.0,0.0,0,1,1,0,0,45-10,11-13,3-0,49-6,7-16,3-1,42-5,12-13,27-16,51-5,47-5,24-7,102.8,95.3,7.5,39-13,16-2,13-5,10-6,20-10,6-4,8-2,6-4,9-8,8-4,12-2,7-1,None,None,None,None,None,2-0,12-4,9-4,36-18,23-5
4,00,22009,1610612737,Atlanta,Hawks,East,32-20,3,- x,Southeast,8-8,2,53,29,0.646,0,53-29,34-7,19-22,7-3,10-0,4-6,4-3,4-6,31-9,12,W 12,-6,L 6,7,4,12,W 12,2,W 2,4,W 4,8.0,6.0,0,0,1,0,0,41-9,11-17,1-3,44-10,7-19,2-0,39-9,13-17,26-18,39-6,39-7,36-16,101.7,97.0,4.7,32-20,13-5,11-7,8-8,21-9,7-3,7-3,7-3,9-6,7-5,11-5,5-3,None,None,None,None,None,2-0,10-5,9-5,33-18,20-11
5,00,22009,1610612756,Phoenix,Suns,West,35-17,3,- x,Pacific,12-4,2,54,28,0.659,0,54-28,32-9,22-19,8-2,9-1,7-3,1-1,5-4,31-10,10,W 10,-7,L 7,10,4,8,W 8,1,W 1,3,W 3,3.0,3.0,0,0,1,0,0,39-10,13-18,2-0,43-5,11-20,0-3,52-18,35-27,26-19,47-10,34-6,19-6,110.2,105.3,4.9,19-11,9-1,5-5,5-5,35-17,11-7,12-4,12-6,7-9,9-3,12-2,5-2,None,None,None,None,None,2-0,12-3,7-9,31-22,23-6
6,00,22009,1610612738,Boston,Celtics,East,33-19,4,- a,Atlantic,13-3,1,50,32,0.610,0,50-32,24-17,26-15,3-7,5-5,5-5,3-1,8-6,23-10,5,W 5,9,W 9,11,3,-2,L 2,-1,L 1,-2,L 2,11.0,0.0,0,1,1,0,0,39-17,11-15,0-0,38-8,8-22,4-2,30-6,14-18,20-23,41-13,27-10,27-13,99.2,95.6,3.7,33-19,13-3,10-8,10-8,17-13,7-3,5-5,5-5,6-8,7-5,11-6,3-5,None,None,None,None,None,3-0,10-4,10-4,32-18,18-14
7,00,22009,1610612743,Denver,Nuggets,West,34-18,4,- nw,Northwest,12-4,1,53,29,0.646,0,53-29,34-7,19-22,6-4,8-2,4-6,4-0,9-4,29-13,9,W 9,-6,L 6,8,3,1,W 1,-1,L 1,-1,L 1,4.0,0.0,0,1,1,0,0,31-10,20-19,2-0,39-5,12-22,2-2,42-10,23-26,33-18,37-3,29-9,29-13,106.5,102.4,4.1,19-11,7-3,7-3,5-5,34-18,12-4,11-7,11-7,12-3,7-5,9-7,5-2,None,None,None,None,None,2-0,10-5,8-7,35-18,18-11
8,00,22009,1610612748,Miami,Heat,East,32-20,5,- x,Southeast,9-7,3,47,35,0.573,0,47-35,24-17,23-18,9-1,7-3,8-2,4-2,7-3,27-18,6,W 6,8,W 8,9,

In [18]:
# list of seasons to pull data from API
seasons = ["2005-06", "2006-07", "2007-08", "2008-09", "2009-10", "2010-11", "2011-12", "2012-13", 
           "2013-14", "2014-15", "2015-16", "2016-17", "2017-18", "2018-19", "2019-20", "2020-21", "2021-22"]

# initialize df with latest full season 21/22
df = endpoints.leaguestandings.LeagueStandings(season="2004-05").standings.get_data_frame()[['SeasonID', 'TeamID', 'TeamCity', 'TeamName', 'Conference', 'PlayoffRank', 'WinPCT', 'DiffPointsPG']]

# concatenate all seasons in list to df
for i in seasons:
    temp_df = endpoints.leaguestandings.LeagueStandings(season=i).standings.get_data_frame()[['SeasonID', 'TeamID', 'TeamCity', 'TeamName', 'Conference', 'PlayoffRank', 'WinPCT', 'DiffPointsPG']]
    df = pd.concat([df, temp_df])

# change seasonid type to numeric
df["SeasonID"] = pd.to_numeric(df["SeasonID"])

df

,SeasonID,TeamID,TeamCity,TeamName,Conference,PlayoffRank,WinPCT,DiffPointsPG
0,22021,1610612748,Miami,Heat,East,1,0.646,4.5
1,22021,1610612756,Phoenix,Suns,West,1,0.780,7.5
2,22021,1610612738,Boston,Celtics,East,2,0.622,7.3
3,22021,1610612763,Memphis,Grizzlies,West,2,0.683,5.7
4,22021,1610612744,Golden State,Warriors,West,3,0.646,5.5
...,...,...,...,...,...,...,...,...
25,22004,1610612757,Portland,Trail Blazers,West,13,0.329,-4.0
26,22004,1610612762,Utah,Jazz,West,14,0.317,-4.3
27,22004,1610612766,Charlotte,Bobcats,East,14,0.220,-6.0
28,22004,1610612737,Atlanta,Hawks,East,15,0.159,-9.7


In [55]:
# playoffrank lag lists that will become columns
PlayoffRankLag1 = []
PlayoffRankLag2 = []
PlayoffRankLag3 = []

# winpct lag lists that will become columns
WinPCTLag1 = []
WinPCTLag2 = []
WinPCTLag3 = []

# diffpointspg lag lists that will become columns
DiffPointsPGLag1 = []
DiffPointsPGLag2 = []
DiffPointsPGLag3 = []


# 2004/05 was the first season NBA had 30 teams
# to get 3 lagged years for each team, final season has to be 2007/08

# loop through df and append 3 lags into lists
for i, row in df.iterrows():
    currentTeamID = row.TeamID
    currentSeasonID = row.SeasonID
    if currentSeasonID >= 22007:
        for n in range(1, 4):
            eval(f'PlayoffRankLag{n}').append(df[(df["SeasonID"] == currentSeasonID - n) & (df["TeamID"] == currentTeamID)].iloc[0]["PlayoffRank"]) # append lagged playoff rank into list
            eval(f'WinPCTLag{n}').append(df[(df["SeasonID"] == currentSeasonID - n) & (df["TeamID"] == currentTeamID)].iloc[0]["WinPCT"]) # append lagged winpct into list
            eval(f'DiffPointsPGLag{n}').append(df[(df["SeasonID"] == currentSeasonID - n) & (df["TeamID"] == currentTeamID)].iloc[0]["DiffPointsPG"]) # append lagged diffpointspg into list
    else:
        for n in range(1, 4):
            eval(f'PlayoffRankLag{n}').append(None)
            eval(f'WinPCTLag{n}').append(None)
            eval(f'DiffPointsPGLag{n}').append(None)

# convert lagged lists into df columns
for n in range(1, 4):
    df[f"PlayoffRankLag{n}"] = eval(f'PlayoffRankLag{n}')
    df[f"WinPCTLag{n}"] = eval(f'WinPCTLag{n}')
    df[f"DiffPointsPGLag{n}"] = eval(f'DiffPointsPGLag{n}')

# remove rows with nan values
df_nonulls = df[df['PlayoffRankLag1'].notna()]

# create target column from playoff rank
df_nonulls['isPlayoff'] = (df_nonulls['PlayoffRank'] <= 8).astype(int)

# split df into east and west conferences
df_east = df_nonulls[df_nonulls["Conference"] == "East"]
df_west = df_nonulls[df_nonulls["Conference"] == "West"]

# drop current playoff rank, winpct, diffpointspg, and conference columns
df_east = df_east.drop(columns=['PlayoffRank', 'Conference', 'WinPCT', 'DiffPointsPG'])
df_west = df_west.drop(columns=['PlayoffRank', 'Conference', 'WinPCT', 'DiffPointsPG'])

C:\Users\mltns\AppData\Local\Temp\ipykernel_15232\1798382739.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nonulls['isPlayoff'] = (df_nonulls['PlayoffRank'] <= 8).astype(int)


In [56]:
df_east

,SeasonID,TeamID,TeamCity,TeamName,PlayoffRankLag1,WinPCTLag1,DiffPointsPGLag1,PlayoffRankLag2,WinPCTLag2,DiffPointsPGLag2,PlayoffRankLag3,WinPCTLag3,DiffPointsPGLag3,isPlayoff
0,22021,1610612748,Miami,Heat,6.0,0.556,0.0,5.0,0.603,2.9,10.0,0.476,-0.2,1
2,22021,1610612738,Boston,Celtics,7.0,0.500,1.5,3.0,0.667,6.3,4.0,0.598,4.4,1
5,22021,1610612749,Milwaukee,Bucks,3.0,0.639,5.9,1.0,0.767,10.1,1.0,0.732,8.9,1
7,22021,1610612755,Philadelphia,76ers,1.0,0.681,5.6,6.0,0.589,2.4,3.0,0.622,2.7,1
8,22021,1610612761,Toronto,Raptors,12.0,0.375,-0.5,2.0,0.736,6.2,2.0,0.707,6.1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20,22007,1610612741,Chicago,Bulls,5.0,0.598,5.0,7.0,0.500,0.6,3.0,0.573,1.1,0
22,22007,1610612766,Charlotte,Bobcats,11.0,0.402,-3.7,13.0,0.317,-4.0,14.0,0.220,-6.0,0
24,22007,1610612749,Milwaukee,Bucks,14.0,0.341,-4.4,8.0,0.481,-1.0,13.0,0.366,-3.0,0
26,22007,1610612752,New York,Knicks,12.0,0.402,-2.9,15.0,0.280,-6.4,12.0,0.402,-2.4,0


In [57]:
df_west

,SeasonID,TeamID,TeamCity,TeamName,PlayoffRankLag1,WinPCTLag1,DiffPointsPGLag1,PlayoffRankLag2,WinPCTLag2,DiffPointsPGLag2,PlayoffRankLag3,WinPCTLag3,DiffPointsPGLag3,isPlayoff
1,22021,1610612756,Phoenix,Suns,2.0,0.708,5.8,10.0,0.466,0.2,15.0,0.232,-9.3,1
3,22021,1610612763,Memphis,Grizzlies,8.0,0.528,1.0,9.0,0.466,-1.1,12.0,0.402,-2.6,1
4,22021,1610612744,Golden State,Warriors,9.0,0.542,1.1,15.0,0.231,-8.7,1.0,0.695,6.5,1
6,22021,1610612742,Dallas,Mavericks,5.0,0.583,2.3,7.0,0.573,4.9,14.0,0.402,-1.3,1
9,22021,1610612762,Utah,Jazz,1.0,0.722,9.3,6.0,0.611,2.5,5.0,0.610,5.3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21,22007,1610612758,Sacramento,Kings,11.0,0.402,-1.8,8.0,0.537,1.5,6.0,0.610,2.2,0
23,22007,1610612746,Los Angeles,Clippers,9.0,0.488,-0.5,6.0,0.573,1.6,10.0,0.451,-0.8,0
25,22007,1610612750,Minnesota,Timberwolves,13.0,0.390,-3.7,14.0,0.402,-1.9,9.0,0.537,1.5,0
27,22007,1610612763,Memphis,Grizzlies,15.0,0.268,-5.1,5.0,0.598,3.7,8.0,0.549,2.3,0


In [ ]:
df_east.to_csv('df_east.csv')
df_west.to_csv('df_west.csv')